In [1]:
from TREE4.TREE4 import *
import pandas as pd
import numpy as np


In [2]:
train = np.genfromtxt("Carseats_train.csv", delimiter=',', names=True, dtype=(float,float,float,float, float, float, object, float, float, object, object))

features_names = train.dtype.names

n_features = train[["ShelveLoc", "US","Urban"]]
n_features_names = n_features.dtype.names
n_features_names

High=[]
for i in train['Sales']:
    if i < 8:
        High.append('NO')
    else:
        High.append('YES')

y = np.array(High)

features = train[["Price", "CompPrice", "Income", "Advertising", "Population", "Age", "Education"]]
features_names = features.dtype.names


indici = np.arange(0, len(y))


In [3]:
test = np.genfromtxt("Carseats_test.csv", delimiter=',', names=True, dtype=(float,float,float,float, float, float, object, float, float, object, object))


n_features_test = test[["ShelveLoc", "US","Urban"]]


High=[]
for i in test['Sales']:
    if i < 8:
        High.append('NO')
    else:
        High.append('YES')


y_test = np.array(High)

features_test = test[["Price", "CompPrice", "Income", "Advertising", "Population", "Age", "Education"]]


In [4]:
#when definining a funcion please be aware we are using purity gain or information gain or greatest difference between variance, all positive aspects 
#adding user_defined as a possible impurity_fn and added user_impur to carry that function 
def user_fn(self, node): #impur just takes node in CART
    
    return (mean(self.y[node.indexes])**2)*len(self.y[node.indexes])

Need to check twoing with more than 2 classes 

In [5]:
my_tree = NodeClass('n1', indici) 
tree = TREE4(y,features,features_names,n_features,n_features_names, impurity_fn = "tau", problem="classifier", method = "FAST",twoing = False,  min_cases_parent= 10,min_cases_child= 5, max_level = 10,  min_imp_gain = 0.01, surrogate_split = False)
tree.growing_tree(my_tree, max_k = 1)

Split Found:  n1 [['ShelveLoc', (b'Medium', b'Bad'), 0.15303824308127323, 0]] start <TREE4.TREE4.NodeClass object at 0x000001AD628B90D0> ShelveLoc in (b'Medium', b'Bad')
node_proportion_total  0.0732358265806532
Split Found:  n2 [['Price', 121.0, 0.1835695169028498, 1]] left <TREE4.TREE4.NodeClass object at 0x000001AD6296E060> Price > 121.0
node_proportion_total  0.3472524878081309
Node_proportion_gain  0.2740166612274777
Split Found:  n4 [['Income', 32.5, 0.16923076923077185, 2]] left <TREE4.TREE4.NodeClass object at 0x000001AD5D5A5640> Income > 32.5
node_proportion_total  0.3803668507389402
Node_proportion_gain  0.03311436293080933
Node Search Split Error for:  n8 with obs in node:  5  which is less than minimum parent size:  10
TypeError: Node search split (CART) failure
This split isn't good now i cut it [counter] - node class purity
TypeError: Node search split (CART) failure
Split Found:  n5 [['Income', 36.0, 0.38095238095238076, 2]] right <TREE4.TREE4.NodeClass object at 0x00000

(((None, None), (None, (None, None))), (None, None))

In [6]:
for i in tree.get_all_node():
    print(i.name, i.local_predictability)
    for j in i.surrogate_splits:
        print(j)

n8 []
n9 []
n10 []
n22 0.257
n23 []
n6 0.333
n7 0.112
n1 0.153
n2 0.184
n4 0.169
n5 0.381
n11 0.234
n3 0.144


In [10]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 


tree.print_tree(table = True, html = False).style.hide() #print both the pruned and non pruned tree 
#cart.print_tree(table = False, html = False, visual_pruning = True)

In [11]:
tree.print_tree(table = False, html = False, merge_leaves = True)

In [12]:
tree.print_tree(table = False, html = False, visual_pruning = True)

In [13]:
tree.print_tree(table = False, html = False, visual_pruning = True, merge_leaves = True )

In [14]:
alpha, pruned_table = tree.pruning(features_test, n_features_test, y_test, table = True)
pruned_table.style.hide()

To return the table, print_tree must be True: setting print_tree to True
Best tree for test set has 5 leaves with misclassification count 32 


In [15]:
tree.print_alpha(alpha)


('Alpha = 0', 'value soglia = None', 'misclassification = 60', 'leaves = 7')
('Alpha = 0.0', "value soglia = ['Income', 32.5, 2]", 'misclassification = 60', 'leaves = 6')
('Alpha = 0.0', "value soglia = ['Education', 15.5, 3]", 'misclassification = 60', 'leaves = 5')
('Alpha = 2.5', "value soglia = ['Price', 121.0, 1]", 'misclassification = 65', 'leaves = 3')
('Alpha = 15.0', "value soglia = ['Price', 92.5, 1]", 'misclassification = 80', 'leaves = 2')


missing value prediction trials 

In [16]:
print(y_test[0])

features_test[0,], n_features_test[0,], y_test[0]


#test = features_test[0,], n_features_test[0,]

new = []
new_n = []
for name in features_names:
    new.append(features_test[name][0])
for n_name in n_features_names:
    new_n.append(n_features_test[n_name][0])    
d = dict(zip(features_names, new))
dn = dict(zip(n_features_names, new_n))
d.update(dn)
d2 = d.copy() 
d2["Price"] = np.nan

d3 = d2.copy()
d3["CompPrice"] = np.nan

tree.pred_x(my_tree, d, tree.get_all_node(), tree.get_leaf())
print(tree.prediction_cat[-1])

tree.pred_x(my_tree, d2, tree.get_all_node(), tree.get_leaf())
print(tree.prediction_cat[-1])

tree.pred_x(my_tree, d3, tree.get_all_node(), tree.get_leaf())
tree.prediction_cat[-1]

NO
NO
NO


'NO'

full dataset model 

In [ ]:
df=pd.read_csv('Carseats.csv')
#df=df.iloc[:,1:]

df.head()

features = df[list(features_names)]
features.head()

n_features = df[list(n_features_names)]
n_features.head()

y = df["Price"]
y.head()

indici = np.arange(0, len(y))


In [ ]:
#my_tree = NodeClass('n1', indici) 

#cart = TREEplus(y,features,features_names,n_features,n_features_names, impurity_fn = "gini", problem="classifier", method = "CART",  min_cases_parent= 10,min_cases_child= 5, max_level = 4)

#cart.growing_tree(my_tree)


need to somehow get only 29 leaves in final tree?
can't prune the traditional way as only have training set 

In [ ]:
#cart.print_tree(table = True, html = True).style.hide_index() #print both the pruned and non pruned tree 

